# 1개

In [ ]:
import cv2
import numpy as np
import torch
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel
from PIL import Image
import os

# ✅ 1️⃣ 그림 생성 (Stable Diffusion ControlNet 사용)
def generate_painting(scribble_prompt_path, text_prompt, save_path, wall_image_path):
    controlnet = ControlNetModel.from_pretrained("lllyasviel/control_v11p_sd15_scribble").to("cuda", torch.float16)
    pipe = StableDiffusionControlNetPipeline.from_pretrained(
        "runwayml/stable-diffusion-v1-5",
        controlnet=controlnet,
        torch_dtype=torch.float16
    ).to("cuda")

    # 🔹 원본 벽 사진 크기 가져오기
    wall_img = Image.open(wall_image_path)
    wall_width, wall_height = wall_img.size  # 원본 벽 사진 크기

    # 🔹 Scribble 이미지 로드 및 크기 조정 (원본 벽 크기와 동일하게 변환)
    scribble_image = Image.open(scribble_prompt_path).convert("L")  # 🔹 흑백 변환
    scribble_image = scribble_image.resize((wall_width, wall_height), Image.LANCZOS)  # 🔹 원본 벽 사진 크기로 조정

    # ✅ 그림 생성 실행 (원본 크기로 Stable Diffusion 설정)
    generated_img = pipe(
        prompt=text_prompt,
        image=[scribble_image],  # ✅ LIST로 전달 (ControlNet 요구 사항)
        controlnet_conditioning_scale=1.0,
        num_inference_steps=50,
        width=wall_width,  # ✅ 원본 벽 크기 반영
        height=wall_height
    ).images[0]

    # ✅ 결과 저장
    generated_img.save(save_path)
    return save_path

# ✅ 2️⃣ 벽 사진에서 각도 추출 (PCA 기반)
def get_rotation_angles(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    _, thresh = cv2.threshold(img, 128, 255, cv2.THRESH_BINARY)
    y, x = np.where(thresh > 0)
    coords = np.column_stack((x, y))

    mean = np.mean(coords, axis=0)
    centered_coords = coords - mean
    cov = np.cov(centered_coords, rowvar=False)
    eigvals, eigvecs = np.linalg.eigh(cov)

    primary_axis = eigvecs[:, 1]
    angle_x = np.arctan2(primary_axis[1], primary_axis[0]) * (180 / np.pi)
    angle_y = np.arctan2(primary_axis[0], primary_axis[1]) * (180 / np.pi)

    print(f"✅ X축 회전 각도: {angle_x:.2f}°")
    print(f"✅ Y축 회전 각도: {angle_y:.2f}°")
    
    return -angle_x, 90-angle_y  # ✅ X축 회전 불필요하면 0 유지

# ✅ 3️⃣ 생성된 그림에 각도 적용
def apply_rotation(image_path, angle_x, angle_y, save_path):
    img = cv2.imread(image_path)
    h, w = img.shape[:2]

    offset_x = np.tan(np.radians(angle_y)) * w / 2
    offset_y = np.tan(np.radians(angle_x)) * h / 2

    src_pts = np.float32([
        [0, 0], [w, 0], [0, h], [w, h]
    ])
    dst_pts = np.float32([
        [offset_x, offset_y], [w - offset_x, -offset_y], 
        [0, h - offset_y], [w, h + offset_y]
    ])

    M = cv2.getPerspectiveTransform(src_pts, dst_pts)
    rotated_img = cv2.warpPerspective(img, M, (w, h))
    cv2.imwrite(save_path, rotated_img)

# ✅ 4️⃣ 각도가 적용된 그림에서 사용자가 제공한 마스크 적용하여 불필요한 부분 삭제
def apply_mask(image_path, mask_path, save_path):
    img = cv2.imread(image_path)
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

    mask_resized = cv2.resize(mask, (img.shape[1], img.shape[0]))
    masked_img = cv2.bitwise_and(img, img, mask=mask_resized)
    
    cv2.imwrite(save_path, masked_img)

# ✅ 5️⃣ 최종적으로 원본 벽 사진에 합성
def merge_with_wall(wall_path, painting_path, save_path):
    wall = cv2.imread(wall_path)
    painting = cv2.imread(painting_path)

    painting_resized = cv2.resize(painting, (wall.shape[1], wall.shape[0]))
    wall[painting_resized > 0] = painting_resized[painting_resized > 0]

    cv2.imwrite(save_path, wall)

# ✅ 6️⃣ 전체 실행 파이프라인 (사용자 제공 마스크 사용)
def process_wall_mural(wall_image, wall_mask, scribble_prompt, text_prompt):
    generated_image_path = "C:/Users/006/Desktop/generation test/output/generated_painting.png"
    rotated_painting_path = "C:/Users/006/Desktop/generation test/output/rotated_painting.png"
    masked_painting_path = "C:/Users/006/Desktop/generation test/output/masked_painting.png"
    final_mural_path = "C:/Users/006/Desktop/generation test/output/final_wall_mural.png"

    # ✅ 그림 생성
    generate_painting(scribble_prompt, text_prompt, generated_image_path, wall_image)

    # ✅ 벽 사진에서 회전 각도 추출
    angle_x, angle_y = get_rotation_angles(wall_image)

    # ✅ 생성된 그림에 동일한 각도 적용
    apply_rotation(generated_image_path, angle_x, angle_y, rotated_painting_path)

    # ✅ 사용자가 제공한 마스크를 적용하여 불필요한 부분 제거
    apply_mask(rotated_painting_path, wall_mask, masked_painting_path)

    # ✅ 원본 벽 사진과 합성하여 최종 벽화 생성
    merge_with_wall(wall_image, masked_painting_path, final_mural_path)

    return final_mural_path, generated_image_path

# ✅ 실행 예시
wall_image = "C:/Users/006/Desktop/generation test/original/1_origin.png"
wall_mask = "C:/Users/006/Desktop/generation test/mask/1_mask.png"
scribble_prompt = "C:/Users/006/Desktop/generation test/sketch/1_sketch.png"  # 스크리블 이미지
text_prompt = "simple drawing, wall painting, ocean with fishes and whales"  # 그림 & 글 프롬프트

final_image, original_generated_image = process_wall_mural(wall_image, wall_mask, scribble_prompt, text_prompt)

print(f"✅ 최종 벽화 이미지: {final_image}")
print(f"✅ 각도 조정 전 생성된 그림: {original_generated_image}")


# 여러개개

In [15]:
import cv2
import numpy as np

def rotate_around_xy_axis(image_path, angle_x=30, angle_y=30, save_path="xy_axis_rotated.png"):
    """
    🔹 X축 & Y축 중심으로 이미지를 회전하여 기울기 적용하는 함수
    - angle_x: X축 중심 기울기 (위/아래 방향)
    - angle_y: Y축 중심 기울기 (좌/우 방향)
    """
    # ✅ 이미지 불러오기
    img = cv2.imread(image_path)
    h, w = img.shape[:2]

    # ✅ 원근 변환을 위한 4개 점 설정
    src_pts = np.float32([
        [0, 0],  # 좌상단
        [w, 0],  # 우상단
        [0, h],  # 좌하단
        [w, h]   # 우하단
    ])

    # ✅ X축과 Y축 기울기를 동시에 적용할 변환 좌표 계산
    offset_x = np.tan(np.radians(angle_y)) * w / 2  # 좌우 기울기
    offset_y = np.tan(np.radians(angle_x)) * h / 2  # 위아래 기울기

    dst_pts = np.float32([
        [offset_x, offset_y],         # 좌상단 이동 (좌/우 + 위/아래 기울기 적용)
        [w - offset_x, -offset_y],    # 우상단 이동 (좌/우 + 위/아래 반대방향)
        [0, h - offset_y],            # 좌하단 이동 (위/아래 반영)
        [w, h + offset_y]             # 우하단 이동 (위/아래 반영)
    ])

    # ✅ 변환 행렬 계산
    M = cv2.getPerspectiveTransform(src_pts, dst_pts)

    # ✅ 변환 적용
    rotated_img = cv2.warpPerspective(img, M, (w, h))

    # ✅ 결과 저장
    cv2.imwrite(save_path, rotated_img)
    print(f"✅ X축 & Y축 중심 회전 완료! 저장 경로: {save_path}")

# 실행 예시
image_path = "C:/Users/006/Desktop/generation test/original/2_origin.png"
save_path = "C:/Users/006/Desktop/generation test/output/xy_axis_rotated.png"
rotate_around_xy_axis(image_path, angle_x=30, angle_y=20, save_path=save_path)


✅ X축 & Y축 중심 회전 완료! 저장 경로: C:/Users/006/Desktop/generation test/output/xy_axis_rotated.png


In [43]:
import cv2
import numpy as np
import torch
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel
from PIL import Image

# ✅ 1️⃣ 그림 생성 (Stable Diffusion ControlNet 사용)
def generate_painting(scribble_prompt_path, text_prompt, save_path, wall_image_path):
    controlnet = ControlNetModel.from_pretrained("lllyasviel/control_v11p_sd15_scribble").to("cuda", torch.float16)
    pipe = StableDiffusionControlNetPipeline.from_pretrained(
        "runwayml/stable-diffusion-v1-5",
        controlnet=controlnet,
        torch_dtype=torch.float16
    ).to("cuda")

    # 🔹 원본 벽 사진 크기 가져오기
    wall_img = Image.open(wall_image_path)
    wall_width, wall_height = wall_img.size  # 원본 벽 사진 크기

    # 🔹 Scribble 이미지 로드 및 크기 조정 (원본 벽 크기와 동일하게 변환)
    scribble_image = Image.open(scribble_prompt_path).convert("L")  # 🔹 흑백 변환
    scribble_image = scribble_image.resize((wall_width, wall_height), Image.LANCZOS)  # 🔹 원본 벽 사진 크기로 조정

    # ✅ 그림 생성 실행 (원본 크기로 Stable Diffusion 설정)
    generated_img = pipe(
        prompt=text_prompt,
        image=[scribble_image],  # ✅ LIST로 전달 (ControlNet 요구 사항)
        controlnet_conditioning_scale=1.0,
        num_inference_steps=50,
        width=wall_width,  # ✅ 원본 벽 크기 반영
        height=wall_height
    ).images[0]

    # ✅ 결과 저장
    generated_img.save(save_path)
    return save_path

# ✅ 2️⃣ 벽 사진에서 각도 추출 (PCA 기반)
def get_rotation_angles(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    _, thresh = cv2.threshold(img, 128, 255, cv2.THRESH_BINARY)
    y, x = np.where(thresh > 0)
    coords = np.column_stack((x, y))

    mean = np.mean(coords, axis=0)
    centered_coords = coords - mean
    cov = np.cov(centered_coords, rowvar=False)
    eigvals, eigvecs = np.linalg.eigh(cov)

    primary_axis = eigvecs[:, 1]
    angle_x = np.arctan2(primary_axis[1], primary_axis[0]) * (180 / np.pi)
    angle_y = np.arctan2(primary_axis[0], primary_axis[1]) * (180 / np.pi)

    print(f"✅ X축 회전 각도: {angle_x:.2f}°")
    print(f"✅ Y축 회전 각도: {angle_y:.2f}°")
    
    return -angle_x, 90-angle_y  # ✅ X축 회전 불필요하면 0 유지

# ✅ 3️⃣ 생성된 그림에 각도 적용
def apply_rotation(image_path, angle_x, angle_y, save_path):
    img = cv2.imread(image_path)
    h, w = img.shape[:2]

    offset_x = np.tan(np.radians(angle_y)) * w / 2
    offset_y = np.tan(np.radians(angle_x)) * h / 2

    src_pts = np.float32([
        [0, 0], [w, 0], [0, h], [w, h]
    ])
    dst_pts = np.float32([
        [offset_x, offset_y], [w - offset_x, -offset_y], 
        [0, h - offset_y], [w, h + offset_y]
    ])

    M = cv2.getPerspectiveTransform(src_pts, dst_pts)
    rotated_img = cv2.warpPerspective(img, M, (w, h))
    cv2.imwrite(save_path, rotated_img)

# ✅ 4️⃣ 각도가 적용된 그림에서 사용자가 제공한 마스크 적용하여 불필요한 부분 삭제
def apply_mask(image_path, mask_path, save_path):
    img = cv2.imread(image_path)
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

    mask_resized = cv2.resize(mask, (img.shape[1], img.shape[0]))
    masked_img = cv2.bitwise_and(img, img, mask=mask_resized)
    
    cv2.imwrite(save_path, masked_img)

# ✅ 5️⃣ 최종적으로 원본 벽 사진에 합성
def merge_with_wall(wall_path, painting_path, save_path):
    wall = cv2.imread(wall_path)
    painting = cv2.imread(painting_path)

    painting_resized = cv2.resize(painting, (wall.shape[1], wall.shape[0]))
    wall[painting_resized > 0] = painting_resized[painting_resized > 0]

    cv2.imwrite(save_path, wall)

# ✅ 6️⃣ 전체 실행 파이프라인 (사용자 제공 마스크 사용)
def process_wall_mural(wall_image, wall_mask, scribble_prompt, text_prompt):
    generated_image_path = "C:/Users/006/Desktop/generation test/output/generated_painting.png"
    rotated_painting_path = "C:/Users/006/Desktop/generation test/output/rotated_painting.png"
    masked_painting_path = "C:/Users/006/Desktop/generation test/output/masked_painting.png"
    final_mural_path = "C:/Users/006/Desktop/generation test/output/final_wall_mural.png"

    # ✅ 그림 생성
    generate_painting(scribble_prompt, text_prompt, generated_image_path, wall_image)

    # ✅ 벽 사진에서 회전 각도 추출
    angle_x, angle_y = get_rotation_angles(wall_image)

    # ✅ 생성된 그림에 동일한 각도 적용
    apply_rotation(generated_image_path, angle_x, angle_y, rotated_painting_path)

    # ✅ 사용자가 제공한 마스크를 적용하여 불필요한 부분 제거
    apply_mask(rotated_painting_path, wall_mask, masked_painting_path)

    # ✅ 원본 벽 사진과 합성하여 최종 벽화 생성
    merge_with_wall(wall_image, masked_painting_path, final_mural_path)

    return final_mural_path, generated_image_path




original_folder = "C:/Users/006/Desktop/generation test/original"
mask_folder = "C:/Users/006/Desktop/generation test/mask"
sketch_folder = "C:/Users/006/Desktop/generation test/sketch"
output_folder = "C:/Users/006/Desktop/generation test/output"
def process_all_images(text_prompt):
    for file_name in os.listdir(original_folder):
        if file_name.endswith((".png", ".jpg", ".jpeg")):
            base_name = os.path.splitext(file_name)[0][0]  # 파일명에서 확장자 제거
            print(base_name)

            # ✅ 각 파일 경로 설정
            wall_image = os.path.join(original_folder, f"{base_name}_origin.png")
            wall_mask = os.path.join(mask_folder, f"{base_name}_mask.png")
            scribble_prompt = os.path.join(sketch_folder, f"{base_name}_sketch.png")
            print(wall_image)
            print(wall_mask)
            print(scribble_prompt)

            # ✅ 해당 파일이 존재하는지 확인
            if not os.path.exists(wall_image) or not os.path.exists(wall_mask) or not os.path.exists(scribble_prompt):
                print(f"⚠️ {base_name}: 원본, 마스크 또는 스케치 파일이 없습니다. 스킵합니다.")
                continue

            print(f"🚀 처리 중: {base_name}")

            # ✅ 결과 저장 경로 설정
            generated_image_path = os.path.join(output_folder, f"{base_name}_generated.png")
            rotated_painting_path = os.path.join(output_folder, f"{base_name}_rotated.png")
            masked_painting_path = os.path.join(output_folder, f"{base_name}_masked.png")
            final_mural_path = os.path.join(output_folder, f"{base_name}_final.png")

            # ✅ 그림 생성
            generate_painting(scribble_prompt, text_prompt, generated_image_path, wall_image)

            # ✅ 벽 사진에서 회전 각도 추출
            angle_x, angle_y = get_rotation_angles(wall_image)

            # ✅ 생성된 그림에 동일한 각도 적용
            apply_rotation(generated_image_path, angle_x, angle_y, rotated_painting_path)

            # ✅ 사용자가 제공한 마스크를 적용하여 불필요한 부분 제거
            apply_mask(rotated_painting_path, wall_mask, masked_painting_path)

            # ✅ 원본 벽 사진과 합성하여 최종 벽화 생성
            merge_with_wall(wall_image, masked_painting_path, final_mural_path)

            print(f"✅ {base_name} 벽화 생성 완료! 저장 경로: {final_mural_path}")

# ✅ 실행 예시 (폴더 내 모든 파일 처리)
text_prompt = "simple drawing, wall painting, ocean with fishes and whales"
process_all_images(text_prompt)

1
C:/Users/006/Desktop/generation test/original\1_origin.png
C:/Users/006/Desktop/generation test/mask\1_mask.png
C:/Users/006/Desktop/generation test/sketch\1_sketch.png
🚀 처리 중: 1


100%|██████████| 50/50 [00:12<00:00,  4.11it/s]


✅ X축 회전 각도: 171.48°
✅ Y축 회전 각도: -81.48°
✅ 1 벽화 생성 완료! 저장 경로: C:/Users/006/Desktop/generation test/output\1_final.png
2
C:/Users/006/Desktop/generation test/original\2_origin.png
C:/Users/006/Desktop/generation test/mask\2_mask.png
C:/Users/006/Desktop/generation test/sketch\2_sketch.png
🚀 처리 중: 2


100%|██████████| 50/50 [00:53<00:00,  1.07s/it]


✅ X축 회전 각도: -179.87°
✅ Y축 회전 각도: -90.13°
✅ 2 벽화 생성 완료! 저장 경로: C:/Users/006/Desktop/generation test/output\2_final.png
3
C:/Users/006/Desktop/generation test/original\3_origin.png
C:/Users/006/Desktop/generation test/mask\3_mask.png
C:/Users/006/Desktop/generation test/sketch\3_sketch.png
🚀 처리 중: 3


...

✅ X축 회전 각도: 90.87°
✅ Y축 회전 각도: -0.87°
✅ 4 벽화 생성 완료! 저장 경로: C:/Users/006/Desktop/generation test/output\4_final.png
5
C:/Users/006/Desktop/generation test/original\5_origin.png
C:/Users/006/Desktop/generation test/mask\5_mask.png
C:/Users/006/Desktop/generation test/sketch\5_sketch.png
🚀 처리 중: 5


100%|██████████| 50/50 [00:03<00:00, 15.48it/s]


✅ X축 회전 각도: -155.40°
✅ Y축 회전 각도: -114.60°
✅ 5 벽화 생성 완료! 저장 경로: C:/Users/006/Desktop/generation test/output\5_final.png
6
C:/Users/006/Desktop/generation test/original\6_origin.png
C:/Users/006/Desktop/generation test/mask\6_mask.png
C:/Users/006/Desktop/generation test/sketch\6_sketch.png
🚀 처리 중: 6


100%|██████████| 50/50 [00:02<00:00, 23.13it/s]


✅ X축 회전 각도: -174.62°
✅ Y축 회전 각도: -95.38°
✅ 6 벽화 생성 완료! 저장 경로: C:/Users/006/Desktop/generation test/output\6_final.png
